[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/enterprise/healthcare/BioNLP_NER.ipynb)

## BioNLP Named Entity Recognition

In [1]:
import json

with open('keys.json') as f:
    license_keys = json.load(f)

license_keys.keys()

dict_keys(['SPARK_NLP_PUBLIC_VERSION', 'SPARK_NLP_VERSION', 'SPARK_NLP_SECRET', 'SPARK_NLP_LICENSE', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'JSL_OCR_LICENSE', 'JSL_OCR_SECRET'])

In [ ]:
import os

# Install java
! apt-get update 
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

secret = license_keys.get("secret",license_keys.get('SPARK_NLP_SECRET', ""))
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['JSL_OCR_LICENSE'] = license_keys['JSL_OCR_LICENSE']
os.environ['AWS_ACCESS_KEY_ID']= license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
version = license_keys.get("version",license_keys.get('SPARK_NLP_PUBLIC_VERSION', ""))
jsl_version = license_keys.get("jsl_version",license_keys.get('SPARK_NLP_VERSION', ""))

! python -m pip install --upgrade spark-nlp-jsl==$jsl_version  --extra-index-url https://pypi.johnsnowlabs.com/$secret

import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

print (sparknlp.version())
print (sparknlp_jsl.version())

spark = sparknlp_jsl.start(secret, gpu=False, spark23=False)

In [ ]:
import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

print (sparknlp.version())
print (sparknlp_jsl.version())

sparknlp_jsl.start(secret, gpu=True, spark23=True)

In [3]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

clinical_pos = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token"]) \
  .setOutputCol("pos")

dependency_parser = DependencyParserModel.pretrained("dependency_conllu") \
  .setInputCols(["sentence","token", "pos"]) \
  .setOutputCol("dependency")

bio_ner = NerDLModel.pretrained('ner_bionlp', 'en', 'clinical/models')

converter = NerConverter()\
  .setInputCols(["sentence", "token", "ner"])\
  .setOutputCol("ner_span")

clinical_ner_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        embeddings,
        clinical_pos,
        dependency_parser,
        bio_ner,
        converter])


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.7 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.6 MB
[OK!]
ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[OK!]


In [4]:
import pyspark.sql.functions as F

text="""
Stage 4 adenocarcinoma of lung with b/l lung nodules , probable L-2 metastasis and a negative brain MRI . Molecular testing of tumor demonstrates EGFR mutation . On Tarceva . Disease Stable . Plan Chemotherapy toxicity reviewed again . Patient agrees to proceed . Counseling time : 40 mins . 1 ) Continue Tarceva for lung cancer . CT CAP ordered before next visit 2 ) Continue anti-anxiety medication 3 ) Xgeva today 4 ) Return in 1 month with labs , after scan
"""

prediction_data = spark.createDataFrame([[text]]).toDF("text")

prediction_data.show(truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------

In [5]:

prediction_model=clinical_ner_pipeline.fit(prediction_data)

preds = prediction_model.transform(prediction_data)

preds.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities")) \
.select(F.expr("entities['0']").alias("chunk"),
        F.expr("entities['1'].entity").alias("entity"))\
        .show(truncate=False)

+--------------+----------------------+
|chunk         |entity                |
+--------------+----------------------+
|adenocarcinoma|Cancer                |
|lung          |Organ                 |
|lung nodules  |Pathological_formation|
|L-2           |Cell                  |
|brain         |Organ                 |
|tumor         |Cancer                |
|EGFR          |Gene_or_gene_product  |
|Tarceva       |Simple_chemical       |
|Patient       |Organism              |
|lung cancer   |Cancer                |
+--------------+----------------------+

